<a href="https://colab.research.google.com/github/deanhoperobertson/Masters-/blob/master/Thesis/Code/Bi_LSTM_CRF_Model_(%2B_Uncased_Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [3]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro.py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-vh0hiify
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-vh0hiify
  Stored in directory: /tmp/pip-ephem-wheel-cache-20e5j6h0/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 51.4MB/s 


In [6]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [8]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [9]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 1


In [10]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 10950


In [11]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [12]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([ 7993, 10950, 15877, 19809, 19232, 18083,  9763, 13709, 21458,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [13]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([1, 5, 2, 5, 5, 5, 2, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

## Adding Embeddings

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:

#root_path = "/content/drive/My Drive/glove.6B.50d.txt"
#root_path = "/content/drive/My Drive/glove.6B.100d.txt"
root_path = "/content/drive/My Drive/glove.6B.200d.txt"
#root_path = "/content/drive/My Drive/wiki-news-300d-1M.vec"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#create emedding matrix

EMBEDDING=200
embedding_matrix = np.zeros((len(word2idx) + 1, EMBEDDING))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [19]:
# Model definition

input = Input(shape=(MAX_LEN,))

model_2 = Embedding(len(word2idx)+1,output_dim=EMBEDDING,
                  weights=[embedding_matrix],input_length=MAX_LEN,
                  trainable=False,mask_zero=True)(input)

#model_2 = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
#                  input_length=MAX_LEN, mask_zero=True)(input)

model_2 = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model_2)

model_2 = TimeDistributed(Dense(10, activation="relu"))(model_2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)

W0730 12:14:10.756337 140328113969024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0730 12:14:10.795193 140328113969024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0730 12:14:10.803917 140328113969024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0730 12:14:10.815787 140328113969024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0730 12:14:10.816795 1403281139

In [20]:
model_2.compile(optimizer='adam', loss=crf_loss)#, metrics=["accuracy"])
model_2.summary()

W0730 12:14:17.447506 140328113969024 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 124, 200)          4856000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 124, 100)          100400    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 124, 10)           1010      
_________________________________________________________________
crf_1 (CRF)                  (None, 124, 6)            114       
Total params: 4,957,524
Trainable params: 101,524
Non-trainable params: 4,856,000
_________________________________________________________________


In [21]:
%%time
BATCH_SIZE = 500
EPOCHS=10

history = model_2.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, verbose=1)

Train on 12636 samples, validate on 1405 samples
Epoch 1/10
12636/12636 [==============================] - 19s 2ms/step - loss: 33.2876 - val_loss: 27.9120
Epoch 2/10
12636/12636 [==============================] - 16s 1ms/step - loss: 33.0026 - val_loss: 27.8075
Epoch 3/10
12636/12636 [==============================] - 17s 1ms/step - loss: 32.8554 - val_loss: 27.6994
Epoch 4/10
12636/12636 [==============================] - 17s 1ms/step - loss: 32.7042 - val_loss: 27.5950
Epoch 5/10
12636/12636 [==============================] - 17s 1ms/step - loss: 32.5920 - val_loss: 27.5303
Epoch 6/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.5282 - val_loss: 27.4915
Epoch 7/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.4905 - val_loss: 27.4664
Epoch 8/10
12636/12636 [==============================] - 17s 1ms/step - loss: 32.4647 - val_loss: 27.4464
Epoch 9/10
12636/12636 [==============================] - 16s 1ms/step - loss: 32.4456 - val_lo

## Predict on Training Set

In [22]:
pred = model_2.predict(X, verbose=1)

14041/14041 [==============================] - 114s 8ms/step


In [23]:
# TRain Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.89      0.85      0.87      8297
        MISC       0.84      0.57      0.68      4593
           O       0.97      0.99      0.98    169578
         ORG       0.84      0.69      0.76     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.93      0.92      0.92     11128

    accuracy                           0.99   1741084
   macro avg       0.91      0.83      0.87   1741084
weighted avg       0.99      0.99      0.99   1741084

0.9945648071204732


In [24]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

         ORG       0.84      0.69      0.76     10025
        MISC       0.84      0.57      0.68      4593
         PER       0.93      0.92      0.92     11128
         LOC       0.89      0.85      0.87      8297

   micro avg       0.88      0.79      0.83     34043
   macro avg       0.87      0.75      0.81     34043
weighted avg       0.88      0.79      0.83     34043

0.8269815969067876


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

In [26]:
pred = model_2.predict(X, verbose=1)

3453/3453 [==============================] - 28s 8ms/step


In [27]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

         ORG       0.83      0.65      0.73      2496
        MISC       0.71      0.56      0.63       918
         PER       0.93      0.88      0.91      2773
         LOC       0.84      0.83      0.84      1925

   micro avg       0.86      0.76      0.81      8112
   macro avg       0.83      0.73      0.78      8112
weighted avg       0.85      0.76      0.80      8112

0.8038866529023513
